# Very First Look at very first ComCam CALEXPBackground Data with DM mosaic 

- Goal verification of early data : Early nights
- Restart the Kernel first
- Applied to CALEXPBackground only
- Status Not working

- LSST Doc for Mosaic : https://pipelines.lsst.io/v/d_2024_06_11/py-api/lsst.pipe.tasks.visualizeVisit.VisualizeMosaicExpTask.html

- author : Sylvie Dagoret-campagne
- affiliation : IJCLab/in2p3/CNRS
- creation date : 2024-10-31
- update : 2024-11-02
- last update : 2025-01-09 : fix the date decoding ix the date decoding (w_2024_50)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.dates as mdates
import matplotlib.ticker
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm
from matplotlib.colors import ListedColormap
from matplotlib import colors

%matplotlib inline

In [ ]:
plt.rcParams["figure.figsize"] = (18,8)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'
plt.rcParams['legend.fontsize']=  12
plt.rcParams['font.size'] = 12

In [ ]:
import numpy as np
import pandas as pd
from astropy.time import Time

In [ ]:
from lsst.daf.butler import Butler 

In [ ]:
# LSST Display
# The advantage to use firefly is that the firefly display can handle a lotsof images
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [ ]:
from lsst.pipe.tasks.visualizeVisit import (
    VisualizeBinExpConfig,
    VisualizeBinExpTask,
    VisualizeMosaicExpConfig,
    VisualizeMosaicExpTask,
)

## Configuration

- Use the tag report to guess the collections in which we exect data :  https://usdf-rsp-dev.slac.stanford.edu/times-square/github/lsst-dm/vv-team-notebooks/TargetReport

In the case of LSSTComCam:

- https://usdf-rsp-dev.slac.stanford.edu/times-square/github/lsst-dm/vv-team-notebooks/TargetReport?day_obs=2024-10-24&instrument=LSSTComCam&repo=embargo_new&collection=LSSTComCam%2Fprompt%2Foutput-2024-10-24&collection_sky=LSSTComCamSim%2Fruns%2FDRP%2FOR4%2Fw_2024_25%2FDM-45066&skymap_name=ops_rehersal_prep_2k_v1&col_sciprog=science_program&col_target=target&col_filter=filter&col_id=id&ts_hide_code=1

### Info from slack channel  #validation_team
day_obs:  2024-10-24  
instrument:  LSSTComCam  
repo:  embargo_new  
collection:  LSSTComCam/prompt/output-2024-10-24  
collection_sky:  LSSTComCamSim/runs/DRP/OR4/w_2024_25/DM-45066  
skymap_name:  ops_rehersal_prep_2k_v1  
col_sciprog:  science_program  
col_target:  target  
col_filter:  filter  
col_id:  id

In [ ]:
repo = 'embargo_new'
instrument = "LSSTComCam"
collection = 'LSSTComCam/prompt/output-2024-10-24'

butler = Butler(repo)

## Query about the collections available

- select collections with LSSTComCam

In [ ]:
for _ in butler.registry.queryCollections():
    if 'LSSTComCam/' in _:
        print(_)

- From the list available today , we probably have to look inside the colleciton `LSSTComCam/nightlyValidation`

## Select the Camera

In [ ]:
# mandatory to get the focal plane setup
camera = butler.get("camera", collections="LSSTComCam/defaults", instrument="LSSTComCam")

In [ ]:
camera

## Select the collection

In [ ]:
#the_collection = "LSSTComCam/runs/nightlyValidation_20241024_20241025/w_2024_42/DM-47059-test" 
#the_collection =   "LSSTComCam/prompt/output-2024-10-24"
the_collection = "LSSTComCam/nightlyValidation"

## Create the butler on the selected collection

In [ ]:
butler = Butler(repo, collections=the_collection)
registry = butler.registry

In [ ]:
for datasetType in registry.queryDatasetTypes():
    if registry.queryDatasets(datasetType, collections= the_collection).any(execute=False, exact=False):
        # Limit search results to the data products
        if ('_config' not in datasetType.name) and ('_log' not in datasetType.name) and ('_metadata' not in datasetType.name) and ('_resource_usage' not in datasetType.name):
            print(datasetType)

- Notice no visit-Table is available. Thus to know which visiId are existing, we will use the registry later

## Extract the list of visitId from the butler's registry 

### Get the list of information that can be extracted from the registryfor each exposure

In [ ]:
print(registry.dimensions["exposure"].RecordClass.fields)

### Create the pandas dataframe from the information inside the butler's registry

In [ ]:
df_exposure = pd.DataFrame(columns=['id', 'obs_id','day_obs', 'seq_num','time_start','time_end' ,'type', 'target','filter','zenith_angle','expos','ra','dec','skyangle','azimuth','zenith','science_program','jd','mjd'])

In [ ]:
where_expr = "instrument='LSSTComCam'"

### Fill the pandas dataframe from the records inside the butler's registry for each exposure

In [ ]:
for count, info in enumerate(registry.queryDimensionRecords('exposure',where = where_expr)):  
    try:
        id_ = info.id
        obs_id_ = info.obs_id
        day_obs_ = info.day_obs
        seq_num_ = info.seq_num
        timespan_ = info.timespan

        timespan_begin_ = pd.to_datetime(info.timespan.begin.to_datetime())
        timespan_end_ = pd.to_datetime(info.timespan.end.to_datetime())
        timespan_jd_ = timespan_.begin.jd
        timespan_mjd_ = timespan_.begin.mjd
        
        
        df_exposure.loc[count] = [info.id, info.obs_id, info.day_obs, info.seq_num,timespan_begin_,timespan_end_ ,info.observation_type, info.target_name, info.physical_filter, info.zenith_angle, \
                             info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle,info.azimuth ,info.zenith_angle, info.science_program,
                             timespan_jd_,timespan_mjd_  ]
    except:
        print(">>>   Unexpected error:", sys.exc_info()[0])
        info_timespan_begin_to_string = "2021-01-01 00:00:00.00"
        info_timespan_end_to_string = "2051-01-01 00:00:00.00"
        info_timespan_begin_jd = 0
        info_timespan_begin_mjd = 0
        df_exposure.loc[count] = [info.id, info.obs_id, info.day_obs, info.seq_num,
                                  pd.to_datetime(info_timespan_begin_to_string),
                                  pd.to_datetime(info_timespan_end_to_string) ,
                                  info.observation_type, info.target_name, 
                                  info.physical_filter, info.zenith_angle, \
                             info.exposure_time,info.tracking_ra, info.tracking_dec, info.sky_angle,info.azimuth ,info.zenith_angle, info.science_program,
                             info_timespan_begin_jd,info_timespan_begin_mjd ]
 

    if count < 2:
        print("-----------------------------------------------------",count,"---------------------------------------------------------")
        print(info)
        print("\t id:                  ",info.id)
        print("\t day_obs:             ",info.day_obs)
        print("\t seq_num:             ",info.seq_num)
        print("\t type-of-observation: ",info.observation_type)
        print("\t target:              ",info.target_name)

        timespan_ = info.timespan
        timespan_begin_ = pd.to_datetime(info.timespan.begin.to_datetime())
        timespan_end_ = pd.to_datetime(info.timespan.end.to_datetime())
        timespan_jd_ = timespan_.begin.jd
        timespan_mjd_ = timespan_.begin.mjd
        
        mjd = timespan_mjd_ 
        jd = timespan_jd_ 
        print("MJD,JD : ",mjd,jd)
        
 

#### convert some columns into integer

In [ ]:
df_exposure = df_exposure.astype({"id": int,'day_obs': int,'seq_num':int})

### Find the different type of exposures

In [ ]:
df_exposure["type"].unique()

### Select the science exposure

In [ ]:
df_science = df_exposure[df_exposure.type == 'science']
df_science.reset_index(drop=True, inplace=True)

In [ ]:
df_science

### Search for calexp and calexpbackground

In [ ]:
dataset_refs_calexp = butler.query_datasets("calexp", collections=the_collection,where="instrument='LSSTComCam'")
dataset_refs_calexpbackground = butler.query_datasets("calexpBackground", collections=the_collection, where="instrument='LSSTComCam'")

In [ ]:
print(len(dataset_refs_calexp),len(dataset_refs_calexpbackground))

In [ ]:
all_visits_withcalexp = []
for idx, ref in enumerate(dataset_refs_calexp ):
    #print(ref.dataId)
    all_visits_withcalexp.append(ref.dataId["visit"])
all_visits_withcalexp = np.sort(np.unique(all_visits_withcalexp))
all_visits_withcalexpbackground = []
for idx, ref in enumerate(dataset_refs_calexpbackground):
    #print(ref.dataId)
    all_visits_withcalexpbackground.append(ref.dataId["visit"])
all_visits_withcalexpbackground = np.sort(np.unique(all_visits_withcalexpbackground))

In [ ]:
ref.dataId

#### are those lists equals ?

In [ ]:
np.array_equal(all_visits_withcalexp ,all_visits_withcalexpbackground)

In [ ]:
print("list of visit with calexp ::",all_visits_withcalexp)

## Example for viewing one image

### Select a date and a visitId

In [ ]:
visitId = 2024102800010 	

### Extract the exposures Calexp from the selected visitID

In [ ]:
def retrieve_images(visitId,instrument=instrument, butler= butler,collection=the_collection,datatype='postISRCCD'):
    the_date = visitId//100000 
    where_clause = f"instrument=\'LSSTComCam\' AND exposure.day_obs={the_date}"
    dataId = {'visit': visitId, 'instrument':instrument}
    
    datasetRefs = butler.registry.queryDatasets(datatype, dataId=dataId, collections  = collection)
    
    
    for i, ref in enumerate(datasetRefs):
        print(ref.dataId)
        exposure = ref.dataId["exposure"]
        detector = ref.dataId["detector"]
        physical_filter = ref.dataId["physical_filter"]
        band = ref.dataId["band"]


    return exposure,band,physical_filter,datasetRefs


In [ ]:
def retrieve_images_calexpbackground(visitId,instrument=instrument, butler= butler,collection=the_collection,datatype='calexpBackground'):
    the_date = visitId//100000 
    where_clause = f"instrument=\'LSSTComCam\' AND exposure.day_obs={the_date}"
    dataId = {'visit': visitId, 'instrument':instrument}
    
    datasetRefs = butler.registry.queryDatasets(datatype, dataId=dataId, collections  = collection)

    image_dict = {}
    
    for i, ref in enumerate(datasetRefs):
        print(ref.dataId)
        visit = ref.dataId["visit"]
        detector = ref.dataId["detector"]
        physical_filter = ref.dataId["physical_filter"]
        band = ref.dataId["band"]
        the_image  = butler.get(ref)
        image_dict[detector]  =  the_image

    return visit,band,physical_filter,datasetRefs,image_dict

In [ ]:
#exposure,band,physical_filter,datasetRefs = retrieve_images(visitId)

### Fetch the calexpbackground references for that visit 

In [ ]:
visit,band,physical_filter,datasetRefs,image_dict = retrieve_images_calexpbackground(visitId)

In [ ]:
list_detidx = list(image_dict.keys())
N = len(list_detidx)
ref_idx = np.argsort(list_detidx)
list_of_images = [ image_dict[ref_idx[idx]] for idx in range(N) ]
list_of_det = [ list_detidx[ref_idx[idx]] for idx in range(N) ]

In [ ]:
list_of_det

## View all the images of the focal surface for that vist

In [ ]:
map_ax_to_det = {0:6,1:7,2:8,3:3,4:4,5:5,6:0,7:1,8:2}
map_det_to_ax = {0:6,1:7,2:8,3:3,4:4,5:5,6:0,7:1,8:2}

In [ ]:
def ShowMosaic(list_of_images,the_title = ""):
    """
    Mosaic for LSSTComCam

    list_of_images : list of LSST images (9 images for LSSTComCam), ordered by detector number 
    """

    N = len(list_of_images)
    assert N== 9
    
    fig,axes = plt.subplots(3,3,figsize=(18,18))
    display = afwDisplay.Display(frame=fig)
    axs = axes.flatten()

    #for idx,ax in enumerate(axs):
    # loop by detector number
    for idx in range(N):
        the_image = list_of_images[idx]
        ax = axs[ map_det_to_ax[idx] ]
        plt.sca(ax)
        #display.scale('asinh', 'zscale')
        img_title = f"det {idx}"
        display.mtv(the_image.getImage(),title=img_title)
        display.show_colorbar(True)
        plt.gca().axis('off')
  
    plt.suptitle(the_title,fontweight="bold",fontsize=25)
    plt.tight_layout()  
    plt.show()

### Plot

In [ ]:
the_title = f"calexpbackground :: id = {visitId}, band = {band}"
ShowMosaic(list_of_images,the_title)

## Loop on all visits

### Select the visits

In [ ]:
#df_sel = df_science[df_science["day_obs"] == 20241030]
#list_of_visitId = list(df_sel.id)
list_of_visitId = np.array(all_visits_withcalexpbackground)
list_of_visitId_nights = np.array([visitId//100000 for visitId in list_of_visitId])

In [ ]:
print("Nights with calexpbackground visits :: ",np.unique(list_of_visitId_nights))

In [ ]:
SELECTED_NIGHT = 20241029
FLAG_SELECT_NIGHT = True
if FLAG_SELECT_NIGHT:
    list_of_visitId = list_of_visitId[list_of_visitId_nights == SELECTED_NIGHT ]
print("list of visit to plot :: ",list_of_visitId) 

### Plot the selected night visits 

In [ ]:
# loop on visit id
Nvis = len(list_of_visitId)
for idcount, visitId in enumerate(list_of_visitId):

    try:
        visit,band,physical_filter,datasetRefs,image_dict = retrieve_images_calexpbackground(visitId)

        list_detidx = list(image_dict.keys())
        N = len(list_detidx)
        ref_idx = np.argsort(list_detidx)
        list_of_images = [ image_dict[ref_idx[idx]] for idx in range(N) ]
        list_of_det = [ list_detidx[ref_idx[idx]] for idx in range(N) ]
    
        the_title = f"{idcount}/{Nvis}) calexpbackground :: id = {visitId}, band = {band}"

        ShowMosaic(list_of_images,the_title)
    except Exception as inst:
        print(inst)   
